In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime, timedelta
import pytz


In [2]:
# Interval required 5 minutes
data = yf.download(tickers='^spx', period='1d', interval='1m')

# declare figure
fig = go.Figure()

# Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                             open=data['Open'],
                             high=data['High'],
                             low=data['Low'],
                             close=data['Close'], name='market data'))

# Add titles
fig.update_layout(
    title='live share price evolution',
    yaxis_title='price')

# Show
fig.show()


[*********************100%***********************]  1 of 1 completed


In [3]:
# data.head()
# for i, row in data.iterrows():
#     print(i.time())

In [10]:
# get buyside and sellside liquidity
start = "2023-06-05"
end = "2023-06-06"
dataForLiquidity = yf.download(tickers='^spx', start=start, end=end, interval='1d')
pSellsideLiquidity = dataForLiquidity.loc[:, "Low"][0]
pBuysideLiquidity = dataForLiquidity.loc[:, "High"][0]
print(f"sellside liquidity: {pSellsideLiquidity}")
print(f"buyside liquidity: {pBuysideLiquidity}")

start = "2023-06-06"
end = "2023-06-07"

# get runs on sellside and buyside
sellsideRuns = []
buysideRuns = []
data = yf.download(tickers='^spx', start=start, end=end, interval='5m')
for i, row in data.iterrows():
    if row["Low"] < pSellsideLiquidity:
        print(f"primary sellside liquidity breached at {i} at {row['Low']}")
        sellsideRuns.append((i, row['Low']))
    if row["High"] > pBuysideLiquidity:
        print(f"primary buyside liquidity breached at {i} at {row['High']}")
        sellsideRuns.append((i, row['High']))

# get swing highs and swing lows
swingHighs = []
swingLows = []
dataForSwings = yf.download(tickers='^spx', start=start, end=end, interval='5m')
for i, row in dataForSwings.iloc[:-2].iterrows():
    preTime = i
    currTime = i + timedelta(minutes=5)
    postTime = i + timedelta(minutes=10)
    # check for swing high
    if dataForSwings.loc[currTime, "High"] > dataForSwings.loc[preTime, "High"] and dataForSwings.loc[currTime, "High"] > dataForSwings.loc[postTime, "High"]:
        print(f"swing high at {currTime} at {dataForSwings.loc[currTime, 'High']}")
        swingHighs.append((i, dataForSwings.loc[currTime, 'High']))
    # check for swing low
    if dataForSwings.loc[currTime, "Low"] < dataForSwings.loc[preTime, "Low"] and dataForSwings.loc[currTime, "Low"] < dataForSwings.loc[postTime, "Low"]:
        print(f"swing low at {currTime} at {dataForSwings.loc[currTime, 'Low']}")
        swingLows.append((i, dataForSwings.loc[currTime, 'Low']))

SEARCH_GREEN_FVG = False
SEARCH_RED_FVG = False

if sellsideRuns and swingHighs:
    SEARCH_GREEN_FVG = True
if buysideRuns and swingLows:
    SEARCH_RED_FVG = True


utc = pytz.UTC

tp1_open = datetime(2023, 6, 6, 10, 0, 0).replace(tzinfo=utc)
tp1_close = datetime(2023, 6, 6, 10, 0, 0).replace(tzinfo=utc)

tp2_open = datetime(2023, 6, 6, 14, 0, 0).replace(tzinfo=utc)
tp2_close = datetime(2023, 6, 6, 15, 0, 0).replace(tzinfo=utc)

intervals = ["5m"]
for interval in intervals:
    dataForFVG = yf.download(tickers='^spx', start=start, end=end, interval=interval)
    if SEARCH_GREEN_FVG:
        for i, row in dataForSwings.iloc[:-2].iterrows():
            preTime = i
            currTime = i + timedelta(minutes=5)
            postTime = i + timedelta(minutes=10)
            if (postTime > tp1_open and postTime < tp1_close) or (postTime > tp2_open and postTime < tp2_close):
                if dataForFVG.loc[postTime, "Low"] - dataForFVG.loc[preTime, "High"] > 0:
                    print(f"FVG Identified at {postTime}")
 
    if SEARCH_RED_FVG:
        for i, row in dataForSwings.iloc[:-2].iterrows():
            preTime = i
            currTime = i + timedelta(minutes=5)
            postTime = i + timedelta(minutes=10)
            if (postTime > tp1_open and postTime < tp1_close) or (postTime > tp2_open and postTime < tp2_close):
                if dataForFVG.loc[postTime, "High"] - dataForFVG.loc[preTime, "Low"] < 0:
                    print(f"FVG Identified at {postTime}")



[*********************100%***********************]  1 of 1 completed
sellside liquidity: 4266.81982421875
buyside liquidity: 4299.27978515625
[*********************100%***********************]  1 of 1 completed
primary sellside liquidity breached at 2023-06-06 09:30:00-04:00 at 4263.5400390625
primary sellside liquidity breached at 2023-06-06 09:35:00-04:00 at 4263.08984375
primary sellside liquidity breached at 2023-06-06 09:40:00-04:00 at 4265.0400390625
primary sellside liquidity breached at 2023-06-06 09:45:00-04:00 at 4266.06982421875
[*********************100%***********************]  1 of 1 completed
swing low at 2023-06-06 09:35:00-04:00 at 4263.08984375
swing high at 2023-06-06 09:50:00-04:00 at 4270.77001953125
swing low at 2023-06-06 09:55:00-04:00 at 4267.490234375
swing high at 2023-06-06 10:10:00-04:00 at 4278.81982421875
swing low at 2023-06-06 10:15:00-04:00 at 4272.64013671875
swing high at 2023-06-06 10:20:00-04:00 at 4278.64013671875
swing high at 2023-06-06 10:30:00